# Time Series Analysis: ARIMA

stationarity: if lags don't depend on time 
check stationarity using hypothesis tests, such as ADF test or KPSS test
many traditional models assume stationarity - which can be achieved through square root or log transforms in some cases

also need to check normality of input vars and output vars

autocorrelation: how data points in different points in time are linearly related to each other as a function of time difference
ACF function: computes correlation between a dataset and the same dataset lagged k = 1, 2, ... , m times
PACF: eliminates some of the redundancies in ACF, effectively providing a more informative succinct summary 
autocorr that falls within a critical region is effectively zero 